In [1]:
from IPython.core.interactiveshell import InteractiveShell
import georasters as gr
import rasterio.mask
import rasterio
import geopandas as gpd
import os
import sys
import glob
import re
import itertools
import collections
import multiprocessing
import requests
import pprint
import pickle
from pathlib import Path
from joblib import delayed, Parallel

# pyscience imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import rioxarray as rxr
from rasterio.merge import merge
import os

/home/ar8787/.conda/envs/geo_shp/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:38: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/ar8787/.conda/envs/geo_shp/lib/python3.11/site-packages/libpysal/cg/alpha_shapes.py:164: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/home/ar8787/.conda/envs/geo_shp/lib/python3.11/site-packages/libpysal/cg

In [2]:
# Main paths
main_path = "/scratch/gpfs/ar8787/groupdata2/india_forest_land/C_Programs/_vcf_10_augsyth/sch/state"
root = Path(main_path)

In [3]:
total_files = 19 

In [6]:
n_sample = 1
for i in np.arange(0, total_files):

    text1 = f"""

library(augsynth)
library(data.table)
library(dplyr)
library(fixest)
library(ggplot2)


# Getting the workign directory 
shell_root <- "/scratch/gpfs/ar8787/groupdata2/india_forest_land" 
dbox_root <- "~/Dropbox/india_forest_land" 
root <- shell_root
setwd( root )


# Getting the num of files
df1 <- fread("A_MicroData/data_sysdif.csv")


# Use tstrsplit to split the 'shrid' column and assign the output to new columns:
df1[, c("pc11_s_id", "pc11_d_id", "pc11_sd_id", "pc11_tv_id") 
    := tstrsplit(shrid, "-", fixed=TRUE)]
cols_to_convert <- c("pc11_s_id", "pc11_d_id", "pc11_sd_id", "pc11_tv_id")
df1[, (cols_to_convert) := lapply(.SD, as.numeric), .SDcols = cols_to_convert]

# Import village
vill_sch <- fread("A_MicroData/vill_sch.csv")
cols_to_convert <- c("pc11_s_id", "pc11_d_id", "pc11_sd_id", "pc11_tv_id")
vill_sch[, (cols_to_convert) := lapply(.SD, as.numeric), .SDcols = cols_to_convert]


df2_inner <- df1 %>% inner_join( vill_sch, 
                           by = c("pc11_s_id", "pc11_d_id", "pc11_sd_id", "pc11_tv_id"))
list_ids = unique(df2_inner$pc11_state_id)
### Geting State names
state_names <- fread( "A_MicroData/state_names_shrug.csv" )
state_id_code <- list_ids[{i}]
st_name <- state_names[ state_names$state_id == state_id_code ]$state_name


df2 = df2_inner[df2_inner$pc11_state_id == state_id_code]
df2$vill_id <- as.numeric( df2$vill_id )

df2_sch <- df2[df2$sch == 1]
df2_nonsch <- df2[df2$sch == 0]

n_sch = dim(df2_sch)[1]
n_nonsch = dim(df2_sch)[1]

#-------------------------------------------------------------------------------
# sch the median
#-------------------------------------------------------------------------------


"""
    
    text2 = """
result <- tryCatch({
  
  # Estimatio Result
  ppool_syn <- multisynth(per_treecover ~ post_ror_data_entry, 
                          vill_id, year, df2_sch )
  
  filename <- paste0( "sch", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_augsyth/sch/by_state", filename)
  saveRDS( ppool_syn, path )
  
  
  # Get Summary result
  ppool_syn_time_summ <- summary( ppool_syn )
  filename <- paste0( "sch", state_id_code, "_sum.RDS")
  path <- file.path("E_Estimates/_vcf_10_augsyth/sch/by_state", filename)
  saveRDS( ppool_syn_time_summ,  path )
  
  
  
  
  # Get the plot
  est_df <- ppool_syn_time_summ$att
  est_df <- est_df[est_df$Level == "Average", ]
  est_df <- est_df[apply(!is.na(est_df), 1, all),]
  
  
  n_df <- df2[t2ev_ror_data_entry %in% est_df$Time][, .N, by = t2ev_ror_data_entry]
  n_df$Time <- n_df$t2ev_ror_data_entry
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "black") +
    geom_point(color = "blue") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to Event", title = "Ror Data Entry at Block Level if more than 50% Villages are Treated") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +  # Dashed vertical line at x = 5
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( st_name ) +
    theme_minimal()
  
  
  filename <- paste0( "sch_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_augsyth/sch/by_state", filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)
  
}, error = function(err) {
  
  # Code to handle the error
  # You can access information about the error in 'err'
  cat("An error occurred:", conditionMessage(err), "\n")
  
  return( 'NA')
})
print(result)





#-------------------------------------------------------------------------------
# nonsch the median
#-------------------------------------------------------------------------------



result <- tryCatch({
  
  
  # Estimatio Result
  ppool_syn <- multisynth(per_treecover ~ post_ror_data_entry, 
                          vill_id, year, df2_nonsch, time_cohort = TRUE, n_leads = 9 )
  
  filename <- paste0( "nonsch", state_id_code, ".RDS")
  path <- file.path("E_Estimates/_vcf_10_augsyth/sch/by_state", filename)
  saveRDS( ppool_syn, path )
  
  
  # Get Summary result
  ppool_syn_time_summ <- summary( ppool_syn )
  filename <- paste0( "nonsch", state_id_code, "_sum.RDS")
  path <- file.path("E_Estimates/_vcf_10_augsyth/sch/by_state", filename)
  saveRDS( ppool_syn_time_summ,  path )
  
  
  
  
  # Get the plot
  est_df <- ppool_syn_time_summ$att
  est_df <- est_df[est_df$Level == "Average", ]
  est_df <- est_df[apply(!is.na(est_df), 1, all),]
  
  
  n_df <- df2[t2ev_ror_data_entry %in% est_df$Time][, .N, by = t2ev_ror_data_entry]
  n_df$Time <- n_df$t2ev_ror_data_entry
  setorder(n_df,Time )
  setorder(est_df,Time )
  
  
  
  # Getting factors
  rect.length <- (max(est_df[,"Estimate"], na.rm = TRUE) - 
                    min(est_df[,"Estimate"], na.rm = TRUE))/2
  scale_fac <- 0.8 * rect.length / ( max(n_df[,"N"]) )
  min_y_lim <- round(min(est_df$lower_bound), 2) * 1.05
  max_y_lim <- round( max(est_df$upper_bound), 2) * 0.95
  
  est_df[,"xmin"] <- est_df[,"Time"] - 0.2
  est_df[,"xmax"] <- est_df[,"Time"] + 0.2
  est_df[,"ymin"] <- min_y_lim
  est_df[,"ymax"] <- est_df[,"ymin"] + ( n_df[,"N"] * scale_fac )
  
  
  # Getting the plot
  p <- ggplot(est_df, aes(x = Time, y = Estimate)) + 
    geom_line(color = "black") +
    geom_point(color = "blue") +
    geom_errorbar(aes(ymin = lower_bound, ymax = upper_bound), 
                  width = 0.2, color = "red") +
    geom_rect(data = est_df, aes(xmin = xmin, xmax = xmax, ymin = ymin, ymax = ymax), 
              fill = "grey70", colour = "grey69", alpha = 0.4, size = 0.2) + 
    coord_cartesian(ylim = c( min_y_lim, max_y_lim ) ) +
    scale_y_continuous(name = "Estimate", 
                       sec.axis = sec_axis(~(.+ (-1*min_y_lim)) * (1/scale_fac), 
                                           name = "Number of Observations")) + 
    labs(y = "ATT", x = "Time to Event", title = "Ror Data Entry at Block Level if more than 50% Villages are Treated") + 
    geom_vline(aes(xintercept = 0), linetype = "dashed", color = "gray") +  # Dashed vertical line at x = 5
    geom_hline(aes(yintercept = 0), linetype = "dashed", color = "gray") + 
    ggtitle( st_name ) +
    theme_minimal()
  
  
  filename <- paste0( "nonsch_", state_id_code, "_plot.png")
  path <- file.path("F_Figures/_vcf_10_augsyth/sch/by_state", filename)
  ggsave(filename = path, plot = p, width = 6, height = 4, dpi = 300)
  
  
  
}, error = function(err) {
  # Code to handle the error
  # You can access information about the error in 'err'
  cat("An error occurred:", conditionMessage(err), "\n")
  
  # Optionally, return a default or custom value
  return( "NA")
})


print(result)


"""
    
    # Saving R scripts
    final_code = text1 + text2
    filename = f"sate_{n_sample}"
    with open( fr"{main_path}/batch/{filename}.R", "w") as f:
        f.write( f"{final_code}")
    
    
    text = f"""#!/bin/bash
#SBATCH --nodes=1                # node count
#SBATCH --ntasks=1               # total number of tasks across all nodes
#SBATCH --cpus-per-task=1        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=200G         # memory per cpu-core (4G per cpu-core is default)
#SBATCH --time=08:00:00          # total run time limit (HH:MM:SS)
#SBATCH --mail-user=futurolos9@gmail.com
#SBATCH --mail-type=BEGIN
#SBATCH --mail-type=END
#SBATCH --mail-type=FAIL
#SBATCH --output={filename}.log

module purge
R CMD BATCH {main_path}/batch/{filename}.R
"""
    
    with open( f"{main_path}/batch/{filename}.sbatch", "w") as f:
            f.write( f"{text}")

    n_sample = n_sample + 1

In [5]:
n1 = n_sample - 1
files = np.arange(1, n_sample).reshape( 1, n1 )
names = ['anzony' ]

text1 = f"""
# submit
cd {main_path}/batch
"""
i = 0
for name in names:
    vlaset = files[ i, :]
    
    text2 = ""
    for val in vlaset:

        if val < n_sample:
            text2 = text2 + f"""
sbatch sate_{val}.sbatch
                    """
            
    with open( fr"{main_path}/anzony.sh", "w") as f:
        f.write( f"{text1 + text2}")
    i = i + 1
